###  Introduction 
Different dynamics have contriuted to continuous evolution of the entertainment industry in recent years. For instance, technology has played a critical role in meeting the high  demand for quality and original video contents. This reality has prompted many movie companies to reinvent themseves in the production, capture and share of quality products to the market. Our company is not left behind on these emerging trends. We are setting up a new movie studio with an objective of capturing the market expectations by producing quality and original  films.  

To succeed in this, just like in any other venjure, we need a workig strategic plan and a comprehensive understanding knowledge of the market trends. First, we need to determine the types of films that are currenly dominating the market and analyze key characteristics and factors that contribute to commercial success.

Our project will explore the major successful films in the industry today and extract actionable insights to inform our movie studio's content strategy. By aligning our creative vision with the market's proven preferences, we aim to maximize our chances of producing hit films that capture the imagination of audiences worldwide.

### Importing Libraries

## Objectives

### 1. What are the audiences preferrences?

### 2. Which movies are most profitable in terms of revenue and profit margin?

### 3. Which genre  had highest ratings?

In [ ]:
rating_df = pd.read_csv("../../DataSets/rt.movie_info.tsv",sep ='\t')

#### Data Cleaning

In [8]:
rating_df.head()

NameError: name 'rating_df' is not defined

In [ ]:
#Checking for duplicates;There are no duplicates in my dataset
duplicated =rating_df.duplicated().sum()
duplicated

In [9]:
#Check for missing values
missing =rating_df.isnull().sum()
missing 

NameError: name 'rating_df' is not defined

In [ ]:
#Fill the NA values in synopsis with No synopsis available
rating_df['synopsis'].fillna("No synopsis available", inplace=True)
#   In rating null =mode
mode_rating = rating_df['rating'].mode()[0]
rating_df['rating'].fillna(mode_rating, inplace=True)
#Replace missing genre with unknown 
rating_df['genre'].fillna("Unknown", inplace=True)
#Replace writing with unknown
rating_df['writer'].fillna("Unknown", inplace=True)
#Drop rows with missing theatre date 
rating_df.dropna(subset=['theater_date'], inplace=True)
#Fill na directors as Unknown
rating_df['director'].fillna("Unknown", inplace=True)



In [ ]:
#Dropping bot currency and box office columns 
rating_df.drop(columns=['currency', 'box_office'], inplace=True)
#Dropping the nulls in runtime colmnn
rating_df.dropna(subset=['runtime'], inplace=True)
#Checking if we have any null values 
rating_df.isnull().sum()

#### Data types

In [ ]:
rating_df.dtypes

In [ ]:
#Converting Data type
#ID to Integer 
rating_df['id'] = rating_df['id'].astype(int)
#Date to datetime
rating_df['theater_date'] = pd.to_datetime(rating_df['theater_date'], errors='coerce')
rating_df['dvd_date'] = pd.to_datetime(rating_df['dvd_date'], errors='coerce')
#Remove text in runtime and converting the data to int 
rating_df['runtime'] = rating_df['runtime'].str.replace(' minutes', '').astype(int)

In [ ]:
rating_df.head()

In [ ]:
#Assighn the rating to numerical values
rating_df["rating"].unique()

In [ ]:
#Mapping 
rating_mapping = {'R': 1, 'PG': 2, 'PG-13': 3, 'NR': 4, 'G': 5, 'NC17': 6}
rating_df['numeric_rating'] = rating_df['rating'].map(rating_mapping)

In [ ]:
rating_counts = rating.value_counts()

### Data Visualization 

In [10]:
#### 10 most popular Movies

In [ ]:
#An Histogram presentation of Genre and the number of movie counts
rating_df['genre'] = rating_df['genre'].fillna('').apply(lambda x: x.split('|'))
genre_counts = rating_df.explode('genre')['genre'].value_counts().head(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=genre_counts.values, y=genre_counts.index)
plt.title('Top 10 Movie Genres')
plt.xlabel('Number of Movies')
plt.ylabel('Genre')
plt.show()

#### Interpretation

-The bar plot shows that the Drama Gernre has a a lot of movies produced followed by the comedy genre


In [11]:
#### A ratings vs top 10 studio plot

This is a plot to show the studio and how the ratings are spread across the top 10 studios

SyntaxError: invalid syntax (3799270843.py, line 3)

In [ ]:
studio_rating_counts = rating_df.groupby(['studio', 'numeric_rating'])

# Plotting the data
top_studios = rating_df['studio'].value_counts().nlargest(10).index
top_studio_data = rating_df[rating_df['studio'].isin(top_studios)]
plt.figure(figsize=(12, 6))
sns.countplot(data=top_studio_data, x='studio', hue='rating')
plt.title("Rating Distribution Across Top 10 Studios")
plt.xlabel("Studio")
plt.ylabel("Count of Ratings")
plt.xticks(rotation=45)
plt.legend(title='Rating')
plt.show()

##### Observation 

The independent Studios hav the highest number of movies produced

##### Chi-Test

I want to do a chi-test to test whether th distribution of rating is independent from the genre 

In [ ]:
# Create the total count of rating and genre
Total_data = pd.crosstab(rating_df['rating'], rating_df['genre'])

chi2_stat, p_value, dof, expected = stats.chi2_contingency(Total_data)

print(f"Chi-square Statistic: {chi2_stat}")
print(f"P-value: {p_value}")
print(f"Degrees of Freedom: {dof}")

alpha = 0.05
if p_value > alpha:
    print("Fail to reject the null hypothesis: Ratings and Genre are independent.")
else:
    print("Reject the null hypothesis: Ratings and Genre are dependent.")

#### Observations
- There is a significance between ratings and genre datasets 
- The ratings dataset is not independent with the genre dataset hence they have influnce on one another 
- The high nuumbr of degree of freedom supports our findings 

#### Checking for normality

We want to check if our ratings follow a normal distribution.We will use the Shapiro-wilk Test


In [ ]:
# Shapiro-Wilk Test
from scipy.stats import shapiro
ratings = (rating_df['numeric_rating'])
stat, p_value = shapiro(ratings)
alpha =0.05 
if p_value > alpha :
    print ("The p-value is greater than 0.05, hence we fail to reject the null hypothesis since we do not have enough evidence."
            "we can conclude that the ratings does not significantly deviate from a normal distribution.")
else:
    print("The p-value is less than 0.05 , hence we reject the null hypothesis at 0.05 level of significance and "
          "conclude that the ratings significantly deviates from the normal distribution")

Our Rating data does not have a normal distribution 


PMF of our Ratings 

In [ ]:
pmf = rating_counts / len(ratings)
pmf

In [ ]:
pmf_df = pmf.reset_index()
pmf_df.columns = ['rating', 'probability']
plt.figure(figsize=(10, 6))
sns.barplot(x='rating', y='probability', data=pmf_df, hue='rating')
plt.title('Probability Mass Function of Ratings')
plt.xlabel('Ratings')
plt.ylabel('Probability')
plt.show()

### Observations
 - From our plot the rating 1(r-restricted) have a more frquent rating  hence are  common movies in the market  
 - The adult only movie 6 (NC17) are less common hence no ratings at all.
 - Our data skews towards the over 18

### 4.How movie release dates vary by month and determine peak release times e.g school holidays, Christmas, Valentines

### 5. How has the average box office revenue changed over time to identify trends that could impact movie production and marketing strategies?

### 6. Which Production Studios and Companies Are Dominating the Box Office?

### 7. Character popularity and group cast impact.Assess the financial impact of major characters and group casts in movies